In [1]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # 특정 GPU에 1GB 메모리만 할당하도록 제한
    try:
        tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
        tf.config.experimental.set_virtual_device_configuration(
            gpus[1],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
    except RuntimeError as e:
    # 프로그램 시작시에 가상 장치가 설정되어야만 합니다
        print(e)

In [ ]:
%reset

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

train_dir = './data/cat_dog_small/train'
validation_dir = './data/cat_dog_small/validation'

# ImageDataGenerator
# 이미지 데이터의 값을 1/225로 scaling
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

# train : 2000개
# validation : 1000개

# train data를 가져오면 => x_data(독립변수,픽셀데이터), t_data(종속변수,label)
train_generator = train_datagen.flow_from_directory(
    train_dir,                # taget directory : 데이터를 어디에서 가져올건가요!
    classes=['cats', 'dogs'], # label적용을 어떻게 할건가요? 순서대로 레이블이 적용
                              # cats : 0, dogs : 1  (cats와 dogs는 폴더명!!)
                              # 만약 classes를 명시하지 않으면 폴더명 순서로 label이 잡혀요!   
    target_size=(150,150),    # 이미지 size scaling(크기를 150x150으로 변경해서 들고와!!)
    batch_size=20,            # 한번에 20개의 이미지를 가져와!!  
                              # label에 상관없이 가져와요!!
    class_mode='binary'       # 이진분류이기 때문에 'binary'
                              # 만약 MNIST처럼 다중분류면 'categorical'
                              # 기본값은 'categorical'
    )

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,           # taget directory : 데이터를 어디에서 가져올건가요!
    classes=['cats', 'dogs'], # label적용을 어떻게 할건가요? 순서대로 레이블이 적용
                              # cats : 0, dogs : 1  (cats와 dogs는 폴더명!!)
                              # 만약 classes를 명시하지 않으면 폴더명 순서로 label이 잡혀요!   
    target_size=(150,150),    # 이미지 size scaling(크기를 150x150으로 변경해서 들고와!!)
    batch_size=20,            # 한번에 20개의 이미지를 가져와!!  
                              # label에 상관없이 가져와요!!
    class_mode='binary'       # 이진분류이기 때문에 'binary'
                              # 만약 MNIST처럼 다중분류면 'categorical'
                              # 기본값은 'categorical'
    )

# 데이터 준비가 끝났어요!!
# 이제 모델을 만들어 보아요!!

with tf.device('/device:GPU:1'):
    
    model = Sequential()
    
    model.add(Conv2D(filters=32,
                     kernel_size=(3,3),
                     activation='relu',
                     input_shape=(150,150,3)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Conv2D(filters=64,
                     kernel_size=(3,3),
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Conv2D(filters=128,
                     kernel_size=(3,3),
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Conv2D(filters=128,
                     kernel_size=(3,3),
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(units=512,
                    activation='relu'))
    
    model.add(Dense(units=1,
                    activation='sigmoid'))
       
    model.compile(optimizer=Adam(learning_rate=1e-4),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    history = model.fit(train_generator,
                        steps_per_epoch=100,    # bach_size * steps_per_epoch > data의 양  XXX
                        epochs=30,
                        validation_data=validation_generator,
                        validation_steps=50)

model.save('./cat_dog_small_cnn_model.h5')    

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Epoch 1/30
100/100 [==============================] - 9s 87ms/step - loss: 0.6928 - accuracy: 0.5140 - val_loss: 0.6791 - val_accuracy: 0.5210
Epoch 2/30
100/100 [==============================] - 9s 86ms/step - loss: 0.6687 - accuracy: 0.5805 - val_loss: 0.6816 - val_accuracy: 0.5500
Epoch 3/30
100/100 [==============================] - 9s 86ms/step - loss: 0.6499 - accuracy: 0.6315 - val_loss: 0.6310 - val_accuracy: 0.6450
Epoch 4/30
100/100 [==============================] - 9s 87ms/step - loss: 0.6105 - accuracy: 0.6650 - val_loss: 0.5968 - val_accuracy: 0.6780
Epoch 5/30
100/100 [==============================] - 8s 84ms/step - loss: 0.5552 - accuracy: 0.7160 - val_loss: 0.5802 - val_accuracy: 0.6880
Epoch 6/30
100/100 [==============================] - 8s 83ms/step - loss: 0.5244 - accuracy: 0.7400 - val_loss: 0.5689 - val_accuracy: 0